# RTS Dataset Formatting

# TODO

-  Make the script find and download the current version of the RTS dataset and metadata description rather than providing the filepath manually
- remove filepaths and metadata columns that I set
-  think through environment sharing (rts_dataset.rproj, .env, renv)
-  finalize file organization/filepaths so that they don't differ between R and Python

# Set-Up

In [1]:
import os
import uuid
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib
import warnings

In [2]:
pd.set_option("display.max_columns", 100)

Workaround for horizontal scrollbars not working in Firefox:

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output {display:flex}</style>"))

## User-Defined Input

Before starting, copy your new shapefile into the input_data directory. Make sure that your shapefile uses EPSG:3413 (WGS 84 / NSIDC Sea Ice Polar Stereographic North) as the coordinate system, otherwise this script will not work.

Provide the file name to the data:

In [4]:
new_data_file = 'rts_dataset_test_polygons_new.shp'  # set this
new_data_filepath = os.path.join(
  '../input_data',
  new_data_file)

Provide the names of any metadata fields in your new file that are not already in the official RTS Data Set (please check the list to ensure that the field has not been included previously):

In [5]:
# Use the format `AbbreviatedName: FullName`, where the FullName should be a human-readable name and the AbbreviatedName should be the ESRI shapefile driver abbreviated version of the the FullName.
# Example:
# new_fields = {'CstmCl1': 'CustomColumn1'}
new_fields = {'CstmCl1' : 'CustomColumn1'}

Have you already created RTS centroid columns, or would you like them to be created within this script? Provide either TRUE, if the columns do not exist yet, or FALSE, if you have already created them:

In [6]:
# Example: 
# calculate_centroid = False
calculate_centroid = False

# Functions

## add_empty_columns

In [7]:
def add_empty_columns(df, column_names):
  
  for name in column_names:
    if name not in df.columns:
      df[name] = pd.NA
      
  return df

## check_intersection_info

In [8]:
def check_intersection_info(df):

  duplicated_uuids = df['UUID'].duplicated()  
  duplicated_uuids = df.loc[duplicated_uuids, 'UUID']
  
  df['int_info_complete'] = (
    (df['Intersections'].isnull()) & (df['SelfIntersectionIndices'].str.len() == 0) |
    (~df['Intersections'].isnull()) & (df['RepeatRTS'].notnull() | df['MergedRTS'].notnull() | df['StabilizedRTS'].notnull() | df['AccidentalOverlap'].notnull()) | 
    (df['SelfIntersectionIndices'].str.len() > 0) & (df['UUID'].isin(duplicated_uuids))
  )
  
  if not df['int_info_complete'].all():
    print(df[~df['int_info_complete']])
    raise Exception('Incomplete intersection information provided. See printed rows.')
  
  print('Intersection information is complete.')

## get_earliest_uuid

Return `UUID` from feature with earliest `BaseMapDate` for features in `new_data` that overlap eachother.

In [9]:
def get_earliest_uuid(df_subset, df):

    uuids = [df_subset['UUID']] + [x for x in df_subset['SelfIntersectionIndices'].split(',')]
     
    df = df[df.UUID.isin(uuids)]
    
    earliest_df = df[df.BaseMapDate == df.BaseMapDate.min()]
    
    return earliest_df.UUID.iloc[0]

## run_formatting_checks

In [10]:
def check_lat(lat):
    correct_type = type(lat[0]) == np.float64
    missing_values = pd.isna(lat).values.any()
    reasonable_values = np.all(lat.between(-180, 180))

    if not correct_type:
        raise ValueError('The CentroidLat column is not numeric. Ensure that latitude is reported as decimal degress in WGS 84.')
    elif missing_values:
        raise ValueError('The CentroidLat column is missing values.')
    elif not reasonable_values:
        raise ValueError('Unexpected values found in the CentroidLat column. Ensure that CentroidLat is listed as decimal degress in WGS 84.')

def check_lon(lon):
    correct_type = type(lon[0]) == np.float64
    missing_values = pd.isna(lon).values.any()
    reasonable_values = np.all(lon.between(-180, 180))

    if not correct_type:
        raise ValueError('The CentroidLon column is not numeric. Ensure that longitude is reported as decimal degress in WGS 84.')
    elif missing_values:
        raise ValueError('The CentroidLon column is missing values.')
    elif not reasonable_values:
        raise ValueError('Unexpected values found in the CentroidLon column. Ensure that longitude is listed as decimal degress in WGS 84.')

def check_region(region):
    correct_type = type(region[0]) == str
    missing_values = (region == '').values.any()

    if not correct_type:
        raise ValueError('The RegionName column is not a string.')
    elif missing_values:
        raise ValueError('The RegionName column is missing values.')

def check_creator(creator):
    correct_type = type(creator[0]) == str
    missing_values = (creator == '').values.any()

    if not correct_type:
        raise ValueError('The CreatorLab column is not a string.')
    elif missing_values:
        raise ValueError('The CreatorLab column is missing values.')

def check_contribution_date(contribution_date):
    correct_type = type(contribution_date[0]) == pd._libs.tslibs.timestamps.Timestamp
    missing_values = (contribution_date == '').values.any()

    if not correct_type:
        raise ValueError('The ContributionDate column is not a string.')
    elif missing_values:
        raise ValueError('The ContributionDate column is missing values.')
  
def check_basemap_date(basemap_date):
    correct_type = pd.Series([
        type(pd.to_datetime(row, dayfirst = True)) == pd.core.indexes.datetimes.DatetimeIndex 
         for row in basemap_date.str.split(',')
    ]).values.all()
    missing_values = ((basemap_date.str.split(',', expand = True)).iloc[:, 0] == '').any()

    if not correct_type:
        raise ValueError('The BaseMapDate column does not contain dates (or they are improperly formatted).')
    elif missing_values:
        raise ValueError('The BaseMapDate column is missing values.')
  
def check_source(source):
    correct_type = type(source[0]) == str
    missing_values = (source == '').values.any()

    if not correct_type:
        raise ValueError('The BaseMapSource column is not a string.')
    elif missing_values:
        raise ValueError('The BaseMapSource column is missing values.')

def check_resolution(resolution):
    correct_type = type(resolution[0]) == np.float64
    missing_values = pd.isna(resolution).values.any()

    if not correct_type:
        raise ValueError('The BaseMapResolution column is not a numeric.')
    elif missing_values:
        raise ValueError('The BaseMapResolution column is missing values.')

def check_train_class(train_class):
    correct_type = type(train_class[0]) == str
    missing_values = (train_class == '').values.any()

    if not correct_type:
        raise ValueError('The TrainClass column is not a string.')
    elif missing_values:
        raise ValueError('The TrainClass column is missing values.')

def run_formatting_checks(df):
    check_lat(df.CentroidLat)
    check_lon(df.CentroidLon)
    check_region(df.RegionName)
    check_creator(df.CreatorLab)
    check_contribution_date(df.ContributionDate)
    check_basemap_date(df.BaseMapDate)
    check_source(df.BaseMapSource)
    check_resolution(df.BaseMapResolution)
    check_train_class(df.TrainClass)
    
    print('Formatting looks good!')


# Import Metadata Description File

In [11]:
### update this to download the current version automatically ##################
col_metadata = pd.read_csv('../input_data/metadata_description.csv')
################################################################################

required_fields = (
    pd.Series(
        col_metadata[col_metadata.Required == True].FullColumnName.values, 
        index = col_metadata[col_metadata.Required == True].AbbreviatedColumnName
    )
    .to_dict()
)
optional_fields = (
    pd.Series(
        col_metadata[col_metadata.Required == False].FullColumnName.values, 
        index = col_metadata[col_metadata.Required == False].AbbreviatedColumnName
    )
    .to_dict()
)

all_fields = dict(required_fields,
                  **optional_fields,
                  **new_fields
                 )

# Import Official and New RTS Data Files

In [12]:
### update this to find the dataset online and download it #####################
rts_data_filepath = os.path.join(
  '../input_data',
  'rts_dataset_test_polygons_current.shp'
  )

rts_data = (
    gpd.read_file(rts_data_filepath)
    .rename(columns = dict(required_fields, **optional_fields))
    .filter(items = list(required_fields.values()) + list(optional_fields.values()) + ['geometry'])
)

rts_data.ContributionDate = pd.to_datetime(rts_data.ContributionDate)
    
for field in list(required_fields.values()): # Check if all required columns are present
    if field not in rts_data.columns:
        raise ValueError('{field} is missing. Has the RTS data set been modified since download?'.format(field = repr(field)))

# for field in list(optional_fields.values()): # Check if all optional columns are present
#     if field not in rts_data.columns:
#         raise ValueError('{field} is missing. Has the RTS data set been modified since download?'.format(field = repr(field)))

rts_data
################################################################################


,CentroidLat,CentroidLon,RegionName,CreatorLab,ContributionDate,BaseMapDate,BaseMapSource,BaseMapResolution,TrainClass,UUID,StabilizedRTS,MergedRTS,geometry
0,68.339180,70.016684,Yamal-Gydan,Rodenhizer,2023-09-01,"01-05-2022,30-09-2022",WorldView-2,4.0,Positive,a6cf10e7-6515-5ab7-b7bb-66606b16ef10,NaN,NaN,"POLYGON ((2007198.307 865988.469, 2007189.916 ..."
1,68.339167,70.016220,Yamal-Gydan,Rodenhizer,2023-09-01,"01-05-2022,30-09-2022",WorldView-2,4.0,Positive,dd137c86-6db6-57da-bb46-961c89845a97,NaN,NaN,"POLYGON ((2007253.161 866032.001, 2007235.776 ..."
2,68.332416,70.016480,Yamal-Gydan,Rodenhizer,2023-09-01,"01-05-2022,30-09-2022",WorldView-2,4.0,Positive,83b6a6ee-b498-5307-ada5-c4558e63a611,NaN,NaN,"POLYGON ((2007310.378 865857.070, 2007340.692 ..."
3,68.329500,70.015499,Yamal-Gydan,Rodenhizer,2023-09-01,"01-05-2022,30-09-2022",WorldView-2,4.0,Positive,6c8a822f-e027-561d-915f-f3cdfc462594,NaN,NaN,"POLYGON ((2007453.557 865845.775, 2007456.723 ..."
4,68.332959,70.014513,Yamal-Gydan,Rodenhizer,2023-09-01,"01-05-2022,30-09-2022",WorldView-2,4.0,Positive,ec6c9c21-1404-552d-8f09-6016f68ed15b,NaN,NaN,"POLYGON ((2007514.965 865926.094, 2007508.132 ..."
5,68.334934,70.014369,Yamal-Gydan,Rodenhizer,2023-09-01,"01-05-2022,30-09-2022",WorldView-2,4.0,Positive,6484485f-b005-58ca-addb-f79829f5a929,NaN,NaN,"POLYGON ((2007496.803 865994.362, 2007488.866 ..."


In [13]:
if calculate_centroid:
    new_data = gpd.read_file(new_data_filepath)

    new_data["CntrdLt"] = new_data.centroid.x
    new_data["CntrdLn"] = new_data.centroid.y

    new_data = (
        new_data    
        .rename(columns = dict(
            {key:required_fields[key] for key in [key for key in list(required_fields.keys()) if key != 'UUID']},
            **optional_fields,
            **new_fields
            )
                )
        .filter(items = list(required_fields.values()) + list(optional_fields.values()) + list(new_fields.values()) + ['geometry'])
        )

    new_data.ContributionDate = pd.to_datetime(new_data.ContributionDate)
    
    for field in [item for item in list(required_fields.values()) if item not in ['UUID']]: # Check if all required columns are present
        if field not in new_data.columns:
            raise ValueError('{field} is missing. Ensure that all required fields (except UUID) are present prior to running this script'
                             .format(field = repr(field)))

    for field in [item for item in list(new_fields.values()) if item not in ['UUID']]: # Check if all new columns are present
        if field not in new_data.columns:
            raise ValueError('{field} is missing. Did you specify the name of the new metadata field correctly?'.format(field = repr(field)))

else:
    
    new_data = (
        gpd.read_file(new_data_filepath)
        .rename(columns = dict(
            {key:required_fields[key] for key in [key for key in list(required_fields.keys()) if key != 'UUID']},
            **optional_fields,
            **new_fields
            )
                )
        .filter(items = list(required_fields.values()) + list(optional_fields.values()) + list(new_fields.values()) + ['geometry'])
        )
    
    new_data.ContributionDate = pd.to_datetime(new_data.ContributionDate)
    
    for field in [item for item in list(required_fields.values()) if item not in ['UUID']]: # Check if all required columns are present
        if field not in new_data.columns:
            raise ValueError('{field} is missing. Ensure that all required fields (except UUID) are present prior to running this script'
                             .format(field = repr(field)))

    for field in [item for item in list(new_fields.values()) if item not in ['UUID']]: # Check if all new columns are present
        if field not in new_data.columns:
            raise ValueError('{field} is missing. Did you specify the name of the new metadata field correctly?'
                             .format(field = repr(field)))

new_data

,CentroidLat,CentroidLon,RegionName,CreatorLab,ContributionDate,BaseMapDate,BaseMapSource,BaseMapResolution,TrainClass,CustomColumn1,geometry
0,68.339261,70.016553,Yamal-Gydan,Rodenhizer,2023-09-28,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007199.012 865984.608, 2007188.217 ..."
1,68.340715,70.015426,Yamal-Gydan,Rodenhizer,2023-09-28,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ..."
2,68.332350,70.016517,Yamal-Gydan,Rodenhizer,2023-09-28,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007340.152 865838.514, 2007326.959 ..."
3,68.331148,70.015305,Yamal-Gydan,Rodenhizer,2023-09-28,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007453.557 865845.775, 2007456.416 ..."
4,68.333416,70.014570,Yamal-Gydan,Rodenhizer,2023-09-28,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007492.587 865949.766, 2007492.342 ..."
5,68.334955,70.014478,Yamal-Gydan,Rodenhizer,2023-09-28,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007479.747 865990.850, 2007472.484 ..."
6,68.340715,70.015426,Yamal-Gydan,Rodenhizer,2023-09-28,"01-05-2022,30-9-2022",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ..."


# Check Metadata Format of New Data

In [14]:
run_formatting_checks(new_data)

Formatting looks good!


# Generate UUIDs

Set seed for UUID generation (R) by concatenating all required metadata columns (except UUID) into a single string

In [15]:
new_data.CentroidLat = np.round(new_data.CentroidLat, 13)
new_data.CentroidLon = np.round(new_data.CentroidLon, 13)
new_data.ContributionDate = new_data.ContributionDate.dt.strftime('%d-%m-%Y')
c = new_data.BaseMapResolution == new_data.BaseMapResolution.astype(int)
new_data.loc[c,'BaseMapResolutionStr'] = new_data.BaseMapResolution.astype(int).astype(str)
new_data.loc[~c,'BaseMapResolutionStr'] = new_data.BaseMapResolution.astype(str)

In [16]:
new_data['seed'] = (
    new_data[[
        'CentroidLat', 
        'CentroidLon', 
        'RegionName', 
        'CreatorLab', 
        'ContributionDate', 
        'BaseMapDate', 
        'BaseMapSource', 
        'BaseMapResolutionStr', 
        'TrainClass'
    ]].apply(
        lambda row: ''.join(row.values.astype(str)),
        axis = 1
    )
)
new_data.seed

0    68.339261104234670.0165529204254Yamal-GydanRod...
1    68.340714676900170.0154263449502Yamal-GydanRod...
2    68.332350277384570.0165169977827Yamal-GydanRod...
3    68.331147672551270.0153050153547Yamal-GydanRod...
4    68.333415505029970.0145699849345Yamal-GydanRod...
5    68.334954611879170.014478196315Yamal-GydanRode...
6    68.340714676900170.0154263449502Yamal-GydanRod...
Name: seed, dtype: object

Generate UUIDs

In [17]:
new_data['UUID'] = [str(uuid.uuid5(uuid.NAMESPACE_DNS, name = seed)) for seed in new_data.seed]
new_data.UUID

0    7aa12b12-bc66-5cb5-aaa4-79ca55c4515a
1    65ad3115-083e-5d5f-a98d-43a49ac220b9
2    4dc81d34-3a76-5bc9-bd05-2c8d59fe15a1
3    077d50c3-550d-5dcc-a5d0-cc82d2ebbae3
4    5c170b46-ded0-582d-be79-293160b0554d
5    5e62e8b5-2489-5b04-9344-601ed863cb55
6    4bf95199-c169-54b1-85c8-f20cc32ac29e
Name: UUID, dtype: object

# Check for Intersections with RTS Data Set

Find intersecting RTS polygons from the official RTS data set and retrieve their UUIDs. Create an empty column for the UUIDs of polygons that have been repeated that will be manually populated.

In [18]:
def get_intersecting_uuids(polygon, df):
    intersections = [','.join(gpd.overlay(polygon, df, how='intersection').UUID_2)]
    return intersections

intersections = []
for idx in range(0,new_data.shape[0]):
    new_intersections = get_intersecting_uuids(new_data.iloc[[idx]], rts_data)
    intersections = intersections + new_intersections
    
new_data['Intersections'] = intersections

overlapping_data = new_data.copy()
overlapping_data = overlapping_data[overlapping_data.Intersections.str.len() > 0]
overlapping_data

if overlapping_data.shape[0] > 0:
    if 'RepeatRTS' not in list(overlapping_data.columns.values):
        overlapping_data['RepeatRTS'] = ['']*overlapping_data.shape[0]
    if 'MergedRTS' not in list(overlapping_data.columns.values):
        overlapping_data['MergedRTS'] = ['']*overlapping_data.shape[0]
    if 'StabilizedRTS' not in list(overlapping_data.columns.values):
        overlapping_data['StabilizedRTS'] = ['']*overlapping_data.shape[0]

    overlapping_data['AccidentalOverlap'] = ['']*overlapping_data.shape[0]

    overlapping_data.to_file(
        os.path.join(
            '../python_output',
            new_data_file.split('.')[0] + "_overlapping_polygons.shp"
        )
    )

/tmp/ipykernel_42026/2964380494.py:26: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  overlapping_data.to_file(


At this point, you will need to manually check all polygons with intersections against the polygons in the official RTS data set in your preferred GIS software and save the output to

In [19]:
os.path.join('../python_output', new_data_file.split('.')[0] + "_overlapping_polygons_edited.shp")

'../python_output/rts_dataset_test_polygons_new_overlapping_polygons_edited.shp'

When possible/necessary, try to find imagery that matches the date of the intersecting polygons - this may require contacting the lab that did the original delineation.

Your job is to inspect each of the polygons listed in the 'Intersections' column compared to the new RTS feature and manually copy and paste the UUIDs from the 'Intersections' column into the 'RepeatRTS', 'StabilizedRTS', 'MergedRTS', or 'AccidentalOverlap' based on the relationship between the two polygons.

- Paste the UUID into the RepeatRTS column when the new RTS feature is the same RTS feature as the RTS feature in the 'Intersections' column, but was delineated at a different point in time, by a different lab at the same point in time, or from different imagery at the same point in time. The RTS feature is the same when it was the result of the same RTS initiation event.

- Paste the UUID into the StabilizedRTS column when the RTS feature in the 'Intersections' column is a stabilized RTS scar as of the date of the imagery used in the new RTS delineations.

- Paste the UUID into the MergedRTS column when multiple RTS features in the 'Intersections' column merged to form the new RTS feature.

- Paste the UUID into the AccidentalOverlap column when inaccuracies in delineation of separate RTS features lead to overlap (e.g. features that are very close to each other and the polygons barely touch). 

When this is done, each of the UUIDs in the Intersections column should have been copied into one (and only one) of the 'RepeatRTS', 'StabilizedRTS', 'MergedRTS', or 'AccidentalOverlap' columns.


# Load Manually Edited File and Join to New Data

Add the 'RepeatRTS', 'StabilizedRTS', and 'MergedRTS' columns that you just edited back into `new_data`.

In [20]:
edited_file = os.path.join('../python_output', new_data_file.split('.')[0] + "_overlapping_polygons_edited.shp")

if os.path.exists(edited_file):
    overlapping_data = (
        gpd.read_file(edited_file)
        .rename(columns = {'UUID': 'UUID',
                           'Intrsct': 'Intersections', 
                           'ReptRTS': 'RepeatRTS', 
                           'MrgdRTS': 'MergedRTS',
                           'StblRTS': 'StabilizedRTS',
                           'AccdntO': 'AccidentalOverlap'
                           })
        .filter(items = ['UUID', 'Intersections', 'RepeatRTS', 'MergedRTS', 'StabilizedRTS', 'AccidentalOverlap'])
        )

    new_data = pd.merge(new_data, 
                        overlapping_data, 
                        how = 'outer',
                        on = ['UUID', 'Intersections'])

    new_data.loc[~new_data.RepeatRTS.isnull(), 'UUID'] = new_data.RepeatRTS[~new_data.RepeatRTS.isnull()]

else:
    new_data['RepeatRTS'] = ['']*new_data.shape[0]
    new_data['MergedRTS'] = ['']*new_data.shape[0]
    new_data['StabilizedRTS'] = ['']*new_data.shape[0]
    new_data['AccidentalOverlap'] = ['']*new_data.shape[0]
    
    warnings.warn("No manually edited file has been imported. This is okay if there were no overlapping polygons, but is a problem otherwise.")

new_data

,CentroidLat,CentroidLon,RegionName,CreatorLab,ContributionDate,BaseMapDate,BaseMapSource,BaseMapResolution,TrainClass,CustomColumn1,geometry,BaseMapResolutionStr,seed,UUID,Intersections,RepeatRTS,MergedRTS,StabilizedRTS,AccidentalOverlap
0,68.339261,70.016553,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007199.012 865984.608, 2007188.217 ...",4,68.339261104234670.0165529204254Yamal-GydanRod...,7aa12b12-bc66-5cb5-aaa4-79ca55c4515a,"a6cf10e7-6515-5ab7-b7bb-66606b16ef10,dd137c86-...",NaN,"a6cf10e7-6515-5ab7-b7bb-66606b16ef10,dd137c86-...",NaN,NaN
1,68.340715,70.015426,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ...",4,68.340714676900170.0154263449502Yamal-GydanRod...,65ad3115-083e-5d5f-a98d-43a49ac220b9,,NaN,NaN,NaN,NaN
2,68.332350,70.016517,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007340.152 865838.514, 2007326.959 ...",4,68.332350277384570.0165169977827Yamal-GydanRod...,83b6a6ee-b498-5307-ada5-c4558e63a611,83b6a6ee-b498-5307-ada5-c4558e63a611,83b6a6ee-b498-5307-ada5-c4558e63a611,NaN,NaN,NaN
3,68.331148,70.015305,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007453.557 865845.775, 2007456.416 ...",4,68.331147672551270.0153050153547Yamal-GydanRod...,077d50c3-550d-5dcc-a5d0-cc82d2ebbae3,6c8a822f-e027-561d-915f-f3cdfc462594,NaN,NaN,6c8a822f-e027-561d-915f-f3cdfc462594,NaN
4,68.333416,70.014570,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007492.587 865949.766, 2007492.342 ...",4,68.333415505029970.0145699849345Yamal-GydanRod...,5c170b46-ded0-582d-be79-293160b0554d,ec6c9c21-1404-552d-8f09-6016f68ed15b,NaN,NaN,NaN,ec6c9c21-1404-552d-8f09-6016f68ed15b
5,68.334955,70.014478,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007479.747 865990.850, 2007472.484 ...",4,68.334954611879170.014478196315Yamal-GydanRode...,6484485f-b005-58ca-addb-f79829f5a929,6484485f-b005-58ca-addb-f79829f5a929,6484485f-b005-58ca-addb-f79829f5a929,NaN,NaN,NaN
6,68.340715,70.015426,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2022,30-9-2022",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ...",4,68.340714676900170.0154263449502Yamal-GydanRod...,4bf95199-c169-54b1-85c8-f20cc32ac29e,,NaN,NaN,NaN,NaN


# Check for Intersections within New RTS Data Set

Intersections within the new data set are assumed to be repeat delineations of the same RTS feature. If this is not true (e.g. if you have delineated an old RTS scar and an active RTS feature on top of it), this code will not assign UUIDs properly. In this case, please get in touch with us to determine how to proceed.

In [21]:
intersections = []
for idx in range(0,new_data.shape[0]):
    new_intersections = get_intersecting_uuids(new_data.iloc[[idx]], new_data.drop([idx]))
    intersections = intersections + new_intersections
    
new_data['SelfIntersectionIndices'] = intersections

new_data.loc[new_data.SelfIntersectionIndices.str.len() > 0, 'UUID'] = (
    new_data[new_data.SelfIntersectionIndices.str.len() > 0]
    .apply(get_earliest_uuid, df = new_data, axis = 1)
)

new_data

,CentroidLat,CentroidLon,RegionName,CreatorLab,ContributionDate,BaseMapDate,BaseMapSource,BaseMapResolution,TrainClass,CustomColumn1,geometry,BaseMapResolutionStr,seed,UUID,Intersections,RepeatRTS,MergedRTS,StabilizedRTS,AccidentalOverlap,SelfIntersectionIndices
0,68.339261,70.016553,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007199.012 865984.608, 2007188.217 ...",4,68.339261104234670.0165529204254Yamal-GydanRod...,7aa12b12-bc66-5cb5-aaa4-79ca55c4515a,"a6cf10e7-6515-5ab7-b7bb-66606b16ef10,dd137c86-...",NaN,"a6cf10e7-6515-5ab7-b7bb-66606b16ef10,dd137c86-...",NaN,NaN,
1,68.340715,70.015426,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ...",4,68.340714676900170.0154263449502Yamal-GydanRod...,4bf95199-c169-54b1-85c8-f20cc32ac29e,,NaN,NaN,NaN,NaN,4bf95199-c169-54b1-85c8-f20cc32ac29e
2,68.332350,70.016517,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007340.152 865838.514, 2007326.959 ...",4,68.332350277384570.0165169977827Yamal-GydanRod...,83b6a6ee-b498-5307-ada5-c4558e63a611,83b6a6ee-b498-5307-ada5-c4558e63a611,83b6a6ee-b498-5307-ada5-c4558e63a611,NaN,NaN,NaN,
3,68.331148,70.015305,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007453.557 865845.775, 2007456.416 ...",4,68.331147672551270.0153050153547Yamal-GydanRod...,077d50c3-550d-5dcc-a5d0-cc82d2ebbae3,6c8a822f-e027-561d-915f-f3cdfc462594,NaN,NaN,6c8a822f-e027-561d-915f-f3cdfc462594,NaN,
4,68.333416,70.014570,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007492.587 865949.766, 2007492.342 ...",4,68.333415505029970.0145699849345Yamal-GydanRod...,5c170b46-ded0-582d-be79-293160b0554d,ec6c9c21-1404-552d-8f09-6016f68ed15b,NaN,NaN,NaN,ec6c9c21-1404-552d-8f09-6016f68ed15b,
5,68.334955,70.014478,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007479.747 865990.850, 2007472.484 ...",4,68.334954611879170.014478196315Yamal-GydanRode...,6484485f-b005-58ca-addb-f79829f5a929,6484485f-b005-58ca-addb-f79829f5a929,6484485f-b005-58ca-addb-f79829f5a929,NaN,NaN,NaN,
6,68.340715,70.015426,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2022,30-9-2022",WorldView-2,4.0,Positive,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ...",4,68.340714676900170.0154263449502Yamal-GydanRod...,4bf95199-c169-54b1-85c8-f20cc32ac29e,,NaN,NaN,NaN,NaN,65ad3115-083e-5d5f-a98d-43a49ac220b9


# Check Completeness of Intersection Information

In [22]:
check_intersection_info(new_data)

Intersection information is complete.


# Final Column Selection

In [25]:
new_data = add_empty_columns(
    new_data, 
    [col for col in list(optional_fields.values()) if col not in ['StabilizedRTS', 'MergedRTS']]
)

new_data = new_data[list(all_fields.values()) + ['geometry']]

new_data

,CentroidLat,CentroidLon,RegionName,CreatorLab,ContributionDate,BaseMapDate,BaseMapSource,BaseMapResolution,TrainClass,UUID,StabilizedRTS,MergedRTS,Area,CustomColumn1,geometry
0,68.339261,70.016553,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,7aa12b12-bc66-5cb5-aaa4-79ca55c4515a,NaN,"a6cf10e7-6515-5ab7-b7bb-66606b16ef10,dd137c86-...",<NA>,NaN,"POLYGON ((2007199.012 865984.608, 2007188.217 ..."
1,68.340715,70.015426,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,4bf95199-c169-54b1-85c8-f20cc32ac29e,NaN,NaN,<NA>,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ..."
2,68.332350,70.016517,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,83b6a6ee-b498-5307-ada5-c4558e63a611,NaN,NaN,<NA>,NaN,"POLYGON ((2007340.152 865838.514, 2007326.959 ..."
3,68.331148,70.015305,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,077d50c3-550d-5dcc-a5d0-cc82d2ebbae3,6c8a822f-e027-561d-915f-f3cdfc462594,NaN,<NA>,NaN,"POLYGON ((2007453.557 865845.775, 2007456.416 ..."
4,68.333416,70.014570,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,5c170b46-ded0-582d-be79-293160b0554d,NaN,NaN,<NA>,NaN,"POLYGON ((2007492.587 865949.766, 2007492.342 ..."
5,68.334955,70.014478,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2023,30-09-2023",WorldView-2,4.0,Positive,6484485f-b005-58ca-addb-f79829f5a929,NaN,NaN,<NA>,NaN,"POLYGON ((2007479.747 865990.850, 2007472.484 ..."
6,68.340715,70.015426,Yamal-Gydan,Rodenhizer,28-09-2023,"01-05-2022,30-9-2022",WorldView-2,4.0,Positive,4bf95199-c169-54b1-85c8-f20cc32ac29e,NaN,NaN,<NA>,NaN,"POLYGON ((2007289.337 866129.959, 2007283.521 ..."


In [26]:
new_data.to_file(os.path.join('../python_output', new_data_file.split('.')[0] + "_formatted.shp"))

/tmp/ipykernel_42026/2726628737.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_data.to_file(os.path.join('../python_output', new_data_file.split('.')[0] + "_formatted.shp"))


Now you are ready to submit

In [27]:
os.path.join('../python_output', new_data_file.split('.')[0] + "_formatted.shp")

'../python_output/rts_dataset_test_polygons_new_formatted.shp'